# Exercise 1: NYC Taxi Dataset Analysis with Unix Tools and SQL

Your name: ______________

This assignment is split in three parts:
 1. In the first part, we will analyze a large data set with UNIX tools.
 2. Next, we will see how the analysis we did would be done in SQL.
 3. Lastly, we included a refresher on more advanced SQL.

### Introduction

As you have seen in the lecture, **UNIX tools** are a handy way to quickly get some insights into data sets. They are **available on almost all systems and don't require any setup**, which is a huge advantage if you think about it! I still use the Unix tools we discussed almost every day.

However, they will only get you so far. That's the reason why we will also look into other, **more practical and specialized** ways to analyze large data sets after the first half of the assignment. For **relational datasets**, like the one we have here, you would use some sort of database. **DuckDB** is especially suitable here because it is a very modern implementation and allows you to run queries directly on raw files without needing to import them into the database. This makes it very handy for this task. You will quickly realize how different and probably more straight-forward it will be. You can use the results from the second part to cross-check with your results from the first-part.

Lastly, we will give you some tasks that require the use of more involved SQL features. This should serve as a refresher for you. If you never touched SQL, it might be a bit trickier, but surely not impossible! You can always reach out for help.

### Assignment Structure
We will use Jupyter notebooks throughout most of the exercises. The reason for this is that this tool is oftentimes used in the data science / data engineering space.

Also, particularly in this assignment, we will use bash commands that we run from this notebook. We do this by marking the whole cell for executing with bash via the `%%bash` magic at the beginning. An alternative is to use `%bash` in front of a single line or use `!` followed by the command.
The same applies for SQL code. In this code, we will use the so-called sql "line magic", that is, after setting up the proper config, you can run SQL in DuckDB by just starting a line with `%sql` followed by the SQL string. If you want to write multi-line SQL, you can end a line with `\`, which means that the line will be concatenated with the following line. Be careful with comments if you are using the trick with `\`!

The following Python packages are required for this assignment:
```
pandas
pyarrow
duckdb
jupysql
duckdb-engine
matplotlib
pillow
```


### The Dataset
We will use a data set that contains taxi drives from New York, which is provided for free by the city of New York. Interestingly, this data set is used in many research works because it (a) is easy to work with (b) contains real-life data and (c) has a decent size. For this exercise, we will only use the data from August 2021.

Documentation of this data set is available at <https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page>. Specifically, have a look at <https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf>.
When you look on the website, you will see several mentions of the "Parquet format". We will not use this for now, but we will learn about it later in the course. For now, we will first convert it to CSV, so that you can manually inspect the file.


In [1]:
import pandas as pd

# Download New York Taxi trip data set and convert it to CSV
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-08.parquet')
df.to_csv('2021_Yellow_Taxi_Trip_Data.csv')

## Example

#### Example Q: How many entries does the "2021 Yellow Taxi Trip Data" have?

In [2]:
%%bash

# Explanation: tail -n +2 can be used to skip the header of the csv file. wc -l counts the lines in the output stream.
tail -n +2 "2021_Yellow_Taxi_Trip_Data.csv" | wc -l

2788757


--------

# Part 1: UNIX tools


## Task
Try to answer the questions in the quiz. You should provide the command that you used to come to your answer together with a short explanation. The command should be a "one-liner" that outputs the correct result. You can either work from the terminal on your computer and copy-paste the solutions into the notebook cells or directly run bash commands in the notebook (see example).

## 1) Outliers

First, we would like to understand more about the extreme values of the data set. An early understanding of this helps further down the road, i.e. if some of the extreme values look unreasonable, they should be filtered out before further processing the data set. Sometimes, they even have to be corrected!


#### Q1.1: What is the trip that had the most expensive `total_amount`? Comment on the value, i.e. is it reasonable?

In [3]:
%%bash
head -n 1 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{print "18th column = "$18}'
#Explanation: Captures the first line of the CSV file (the header) and stores it in the variable header
header=$(head -n 1 2021_Yellow_Taxi_Trip_Data.csv)
#tail -n +2 2021_Yellow_Taxi_Trip_Data.csv: This command skips the header line and retrieves all subsequent lines from the CSV file. 
#sort -t, -k18 -nr:This sorts the data numerically (-n) and in reverse order (-r) based on the 18th column (-k18), which is expected to be total_amount. The -t, specifies that the delimiter is a comma.
#head -n 1:This retrieves the first line of the sorted output, which corresponds to the trip with the highest total amount.
#awk -F, '{printf("total_amount = %.2f\n", $18)}': This processes the selected line using awk, setting the field separator to a comma (-F,). It formats and prints the value of the 18th field ($18), which is the total_amount, in the format total_amount = XX.XX.
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | sort -t, -k18 -nr | head -n 1 | awk -F, '{printf("total_amount = %.2f\n", $18)}'

tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{print $18}' | sort -n | tail -n 1

#The total amount of $959.31 for the trip is not reasonable.
#Reasoning:
#-Typical fares for a trip of 12.4 miles in NYC generally range from $30 to $70.
#-This fare is significantly higher than what would be expected, indicating it may be an outlier or a result of an error in data entry. 
#-Further investigation is warranted to understand the cause of this high amount.

18th column = total_amount
total_amount = 959.31
959.31


#### Q1.2: What's the distance of the longest and shortest trip in kilometers? Comment on the value, i.e. is it reasonable? Try to come up with a one-liner that outputs both numbers in one line!  
_Note: The distances in the file are in miles. Try to convert them into kilometers inside your command._

In [4]:
%%bash
head -n 1 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{print "6th column = "$6}'
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F, '{dist_km=$6 * 1.60934; if (dist_km > 0) {if (min == "" || dist_km < min) min=dist_km; if (dist_km > max) max=dist_km}} END {if (min && max) printf "Shortest trip: %.2f km, Longest trip: %.2f km\n", min, max; else print "No valid trip distances found."}' 

#Explanation :
#tail -n +2: Skips the header of the CSV file.
#awk -F, '{...}': Converts the distance from miles to kilometers,Only considers trips where the distance is greater than zero.Updates the minimum (min) and maximum (max) distances accordingly.
#END {...}:Prints the shortest and longest trip distances if valid trips were found.If no valid distances exist, it outputs a message indicating this.

#Comment on the values: 
#Shortest Trip: 0.02 km:
#A shortest trip of 0.02 km (about 20 meters) is reasonable for short urban rides, such as quick pickups or drops within a single block. 
# This indicates that some passengers are indeed taking very short taxi rides, which is common in busy city environments.
#Longest Trip: 495577.87 km:
#A longest trip of 495,577.87 km is not reasonable. This distance is equivalent to more than 12 times the circumference of the Earth, indicating a likely data error or anomaly. Such a trip is practically impossible and suggests either an incorrect entry or a significant issue in the dataset.

6th column = trip_distance
Shortest trip: 0.02 km, Longest trip: 495577.87 km


#### Q1.3: What is the maximum number of passengers on a single taxi ride? Comment on the value, i.e. is it reasonable?

In [5]:
%%bash
head -n 1 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{print "5th column = "$5}'
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F, 'BEGIN {max=0} {if ($5+0 > max) max=$5+0} END {print "Maximum number of passengers:", max}'
#Explanation:
#tail -n +2 skips the header and processes the rest of the file.
#awk -F, processes each line with a comma as the separator.
#BEGIN {max=0} initializes a variable to track the maximum.
#{if ($5+0 > max) max=$5+0} checks if the current value in the fifth column exceeds the current maximum and updates it if so.
#END {print ...} outputs the maximum number of passengers found.

#Comment on the Value
#The maximum of 112 passengers in a single taxi ride seems unreasonable given that a standard yellow taxi in NYC typically accommodates up to 4 passengers. 
#This value could indicate data errors, such as incorrect entries or misreported counts, rather than a realistic scenario in a typical taxi setting. 
#It's worth investigating the dataset for potential anomalies or outliers.

5th column = passenger_count
Maximum number of passengers: 112


#### Q1.4: How many entries do *not* belong in August 2021, i.e. when a trip neither has pickup_datetime nor dropoff_datetime in August 2021?

In [6]:
%%bash

# Check if dates are within August 2021
head -n 1 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{print "3th column = "$3}'
head -n 1 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{print "4th column = "$4}'
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{if ($3 !~ /^2021-08/ && $4 !~ /^2021-08/) count++} END {print "Entries not in August 2021:", count}'

#Explanation:
#tail -n +2 skips the header and processes the rest of the file.
#awk -F',' sets the comma as the field separator.
#The condition checks if the first column (pickup_datetime) and second column (dropoff_datetime) do not start with "2021-08" (indicating August 2021).
#If neither date is in August 2021, it increments the count.
#Finally, it prints the total count of such entries.

3th column = tpep_pickup_datetime
4th column = tpep_dropoff_datetime
Entries not in August 2021: 476


## 2) Missing Values

Next, we'll look at missing values. Missing values might be indicated in various different ways, e.g. NULL, NIL, None, Undefined, -1, empty strings, ...

The column `passenger_count` has quite a lot of missing values.

#### Q2.1: How many missing values are there in the `passenger_count` column?

In [7]:
%%bash
head -n 1 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{print "5th column = "$5}'
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{if ($5 == "" || $5 == "NULL" || $5 == "NIL" || $5 == "None" || $5 == "Undefined" || $5 == "-1") count++} END {print "Missing values in passenger_count:", count}'

5th column = passenger_count
Missing values in passenger_count: 135590


#### Q2.2: Describe how you figured out what value is considered "empty"? Is the data set using NULL, -1 or something different? Come up with a clever way using UNIX tools!

In [8]:
%%bash

# Display the values in the passenger_count column to understand empty representation
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{print $5}' | sort | uniq -c | sort -nr

#Another way
echo "### Summary of passenger_count Analysis"
echo ""
echo "#### Total Entries:"
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | wc -l | awk '{print "Total Count of Records: "$1}'
echo ""
echo "#### Unique Values and Counts:"
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{print $5}' | sort | uniq -c | sort -nr | awk 'BEGIN {print "| Count | Passenger Count |"; print "|-------|------------------|"} {printf("| %-5s | %-16s |\n", $1, ($2 == "" ? "(missing)" : $2))}'
echo ""
echo "#### Findings:"
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{if ($5 == "") count++} END {print "Count of Missing Values: " count}'
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{if ($5 == "0.0") zero++} END {print "Count of 0.0 Entries: " zero}'
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{if ($5 > 6) high++} END {print "Count of High Values (>6): " high}'
echo ""
echo "### Conclusion:"
echo "This analysis identifies significant missing values and unusual passenger counts in the passenger_count column."


1911176 1.0
 403250 2.0
 135590 
 110379 3.0
  72888 0.0
  68864 5.0
  46898 4.0
  39692 6.0
     10 7.0
      7 8.0
      1 96.0
      1 9.0
      1 112.0
### Summary of passenger_count Analysis

#### Total Entries:
Total Count of Records: 2788757

#### Unique Values and Counts:
| Count | Passenger Count |
|-------|------------------|
| 1911176 | 1.0              |
| 403250 | 2.0              |
| 135590 | (missing)        |
| 110379 | 3.0              |
| 72888 | 0.0              |
| 68864 | 5.0              |
| 46898 | 4.0              |
| 39692 | 6.0              |
| 10    | 7.0              |
| 7     | 8.0              |
| 1     | 96.0             |
| 1     | 9.0              |
| 1     | 112.0            |

#### Findings:
Count of Missing Values: 135590
Count of 0.0 Entries: 72888
Count of High Values (>6): 20

### Conclusion:
This analysis identifies significant missing values and unusual passenger counts in the passenger_count column.


## 3) Distributions

Another important piece of information about a data set is a notion of distribution. Data sets might be "top-heavy", i.e. a significant portion of data points are concentrated towards the higher end of the scale, or "bottom-heavy", which is just the opposite of top-heavy. When it comes to heavily skewed data distributions, it's important to understand that single metrics like a MEAN don't fully describe the "average characteristics". A property of top-heavy data sets is that the MEAN is larger than the MEDIAN because there are some large values that pull up the MEDIAN. As we will see in a minute, `passenger_count` is a bottom-heavy distribution.




Let's have a closer look at the `passenger_count` column in this exercise. Only use UNIX tools that are installed per default. No Python scripting, datamash or whatever ;)

In [9]:
%%bash
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{if ($5 + 0 > 0) {sum += $5; count++}} END {if (count > 0) print "Mean passenger count:", sum/count; else print "No valid entries found."}'

#Explanation:
#tail -n +2: Skips the header row to focus on the data.
#awk -F',': Sets the field separator to a comma.
#if ($5 + 0 > 0): Ensures that only valid passenger counts (greater than 0) are considered for the mean calculation.
#sum += $5: Accumulates the sum of the valid passenger counts.
#count++: Increments the count of valid entries.
#END Block: After processing all entries, it calculates and prints the mean if there are valid counts.

Mean passenger count: 1.48016


#### Q3.2: What is the median value for the number of passengers?

_Note: This one is surprisingly tricky. You can assume that you have the number of entries in the data set. Extra points if manage to compute it without any assumptions about the size of the data set._

In [10]:
%%bash

tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{if ($5 + 0 > 0) print $5}' | sort -n | awk '{
    count[NR] = $1
} END {
    n = NR
    if (n % 2 == 1) {
        median = count[int(n / 2)]
    } else {
        median = (count[n / 2 - 1] + count[n / 2]) / 2
    }
    print "Median passenger count:", median
}'

#Explanation:
#Extract Valid Counts: The first awk command extracts valid passenger counts (greater than 0) from the fifth column.
#Sort the Values: The sort -n command sorts the values numerically.
#Store in Array: The second awk command stores the sorted counts in an array.
#Calculate Median:
#It checks if the number of entries (NR) is odd or even.
#It calculates the median based on the number of entries.
#Output: Finally, it prints the median passenger count.

Median passenger count: 1.0


#### Q3.3: What is the p95 value for passenger count?

In [11]:
%%bash
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{if ($5 + 0 > 0) print $5}' | sort -n | awk '{
    count[NR] = $1
} END {
    n = NR
    p95_index = int(n * 0.95) - 1  # 0-based index for 95th percentile
    if (p95_index >= 0) {
        print "p95 passenger count:", count[p95_index]
    } else {
        print "No valid entries found."
    }
}'

#Explanation:
#Extract Valid Counts: The first awk command extracts valid passenger counts (greater than 0) from the fifth column.
#Sort the Values: The sort -n command sorts these values numerically.
#Store in Array: The second awk command stores the sorted counts in an array.
#Calculate p95:
#It calculates the index for the 95th percentile using int(n * 0.95) - 1 to convert it to a zero-based index.
#It checks if the index is valid and retrieves the corresponding value.
#Output: Finally, it prints the p95 passenger count.

p95 passenger count: 4.0


#### Q3.4 How many distinct values (NDV) are there for the `passenger_count` column?

In [12]:
%%bash
tail -n +2 2021_Yellow_Taxi_Trip_Data.csv | awk -F',' '{if ($5 + 0 >= 0) print $5}' | sort -nu | wc -l

#Explanation:
#tail -n +2: Skips the header row to process only the data rows.
#awk -F',' '{if ($5 + 0 >= 0) print $5}': Extracts the fifth column (passenger count) and prints only valid counts (including 0).
#sort -nu: Sorts the values numerically and removes duplicates (-n for numeric sorting, -u for unique).
#wc -l: Counts the number of lines, which corresponds to the number of distinct values

12


-------
# Part 2: Beyond Bash: SQL

You might have noticed that UNIX commands can get pretty complex and are sometimes hard to understand. Also, their performance is sometimes not the best.
In the second part of this exercise you get the chance to do the same thing again, but in SQL. For this, we use DuckDB inside this notebook.

To use DuckDb, you first have to install it (see also https://duckdb.org/docs/guides/python/jupyter.html):
```
pip install duckdb
pip install jupysql
pip install duckdb-engine
pip install pandas
pip install matplotlib
```

If you installed everything correctly, the following cell should just work:

In [13]:
import duckdb
import pandas as pd

# Enable using '%sql' syntax in notebooks:
%reload_ext sql
# Initialize DuckDB
%sql duckdb:///:memory:
# Set output format options via pandas for DuckDB
pd.set_option('display.max_columns', None)

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Connecting to 'duckdb:///:memory:'

In [14]:
# Test that everything works by running a test-query on the input file using DuckDB.
%sql select count(*) as cnt from '2021_Yellow_Taxi_Trip_Data.csv';

,cnt
0,2788757


#### Q4.1: What is the trip that had the most expensive `total_amount`?

In [15]:
# Answer:

%sql SELECT MAX(total_amount) FROM '2021_Yellow_Taxi_Trip_Data.csv';


,max(total_amount)
0,959.31


In [16]:
# Cell is used for grading. Do not remove.

#### Q4.2: What's the distance of the longest and shortest trip in kilometers?

In [3]:
%sql SELECT MAX(trip_distance) * 1.60934 AS longest_trip_km, MIN(trip_distance) * 1.60934 AS shortest_trip_km FROM '2021_Yellow_Taxi_Trip_Data.csv' WHERE trip_distance > 0;

UsageError: Line magic function `%sql` not found.


In [18]:
# Cell is used for grading. Do not remove.

#### Q4.3: What is the maximum number of passengers on a single taxi ride?

In [19]:
# Answer:

%sql SELECT MAX(passenger_count) FROM '2021_Yellow_Taxi_Trip_Data.csv';

,max(passenger_count)
0,112.0


In [20]:
# Cell is used for grading. Do not remove.

    #### Q4.4: How many entries do not belong in August 2021, i.e. when a trip neither has pickup_datetime nor dropoff_datetime in August 2021?

In [21]:
# Answer:
%sql SELECT COUNT(*) AS non_august_entries FROM "2021_Yellow_Taxi_Trip_Data.csv" WHERE NOT ((tpep_pickup_datetime >= '2021-08-01' AND tpep_pickup_datetime < '2021-09-01') OR (tpep_dropoff_datetime >= '2021-08-01' AND tpep_dropoff_datetime < '2021-09-01'));

,non_august_entries
0,476


In [22]:
# Cell is used for grading. Do not remove.

#### Q4.5: How many missing values are there in the `passenger_count` column?

In [23]:
# Answer:

%sql SELECT COUNT(*) FROM '2021_Yellow_Taxi_Trip_Data.csv' WHERE passenger_count IS NULL;


,count_star()
0,135590


In [24]:
# Cell is used for grading. Do not remove.

#### Q4.6: What are the mean, median, p95, and number of distinct values (NDV) for the number of passengers? Return all of the numbers in just one query.

In [25]:
# Answer:
%sql WITH sorted_passenger_counts AS (SELECT passenger_count, ROW_NUMBER() OVER (ORDER BY passenger_count) AS rn, COUNT(*) OVER () AS total FROM '2021_Yellow_Taxi_Trip_Data.csv') SELECT AVG(passenger_count) AS mean, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY passenger_count) AS median, PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY passenger_count) AS p95, COUNT(DISTINCT passenger_count) AS ndv FROM sorted_passenger_counts;

,mean,median,p95,ndv
0,1.439495,1.0,4.0,12


In [26]:
# Cell is used for grading. Do not remove.

-------
# Part 3: More advanced SQL

With SQL, we can do some more advanced analytics as well. This exercise will give you the chance to refresh and brush up your SQL skills.

The following tasks require more features that SQL provides, e.g. joins, aggregations, ...

#### Q5.1: What was the day when the most money was earned by drivers?
At what day in August 2021 was the accumulated money that was spent by customers the highest?

_Note: Think about whether pickup or dropoff time should be used._

In [27]:
# Answer:
%sql SELECT CAST(tpep_pickup_datetime AS DATE) AS day, SUM(total_amount) AS total_amount FROM '2021_Yellow_Taxi_Trip_Data.csv' WHERE tpep_pickup_datetime >= '2021-08-01' AND tpep_pickup_datetime < '2021-09-01' GROUP BY day ORDER BY total_amount DESC LIMIT 1;

,day,total_amount
0,2021-08-20,2.030784e+06


In [28]:
# Cell is used for grading. Do not remove.

#### Q5.2: What are the most popular pickup locations in the morning (8am - 10am) and in the evening (5pm-7pm)

In [29]:
# Answer:
%sql SELECT PULocationID, COUNT(*) AS trip_count FROM '2021_Yellow_Taxi_Trip_Data.csv' WHERE (tpep_pickup_datetime >= '2021-08-01 08:00:00' AND tpep_pickup_datetime < '2021-08-01 10:00:00') GROUP BY PULocationID ORDER BY trip_count DESC LIMIT 1;

%sql SELECT PULocationID, COUNT(*) AS trip_count FROM '2021_Yellow_Taxi_Trip_Data.csv' WHERE (tpep_pickup_datetime >= '2021-08-01 17:00:00' AND tpep_pickup_datetime < '2021-08-01 19:00:00') GROUP BY PULocationID ORDER BY trip_count DESC LIMIT 1;


,PULocationID,trip_count
0,132,598


In [30]:
# Cell is used for grading. Do not remove.

#### Q5.3: Which trips might be trips to and from work? (challenging)
Figure out which pair of trips might be people taking a taxi to and from work. The output should include all pairs of potentially matching trips. Specifically, this might mean that one "trip to work" might be matched with multiple "trip from work".

_Hint: This requires a huge join. Make reasonable assumptions, e.g. trips should be shorter than 60 mins, to reduce the amount of data that will be joined and potentially introduce some preparational steps to speed up processing. For the latter, have a look at `with` expressions in SQL!_

In [31]:
import duckdb

# Create a DuckDB in-memory database
con = duckdb.connect()

# Load the CSV into a DuckDB table
con.execute("""
CREATE TABLE taxi_trips AS 
SELECT 
    tpep_pickup_datetime,
    tpep_dropoff_datetime,
    PULocationID,
    DOLocationID,
    trip_distance,
    passenger_count 
FROM read_csv_auto('2021_Yellow_Taxi_Trip_Data.csv')
""")

# Execute the SQL query in one line
result = con.execute("WITH to_work AS (SELECT tpep_pickup_datetime AS pickup_to_work, tpep_dropoff_datetime AS dropoff_to_work, PULocationID AS pickup_location_to_work, DOLocationID AS dropoff_location_to_work FROM taxi_trips WHERE trip_distance > 0 AND passenger_count > 0 AND (tpep_dropoff_datetime - tpep_pickup_datetime) < INTERVAL '1 hour'), from_work AS (SELECT tpep_pickup_datetime AS pickup_from_work, tpep_dropoff_datetime AS dropoff_from_work, PULocationID AS pickup_location_from_work, DOLocationID AS dropoff_location_from_work FROM taxi_trips WHERE trip_distance > 0 AND passenger_count > 0 AND (tpep_dropoff_datetime - tpep_pickup_datetime) < INTERVAL '1 hour') SELECT tw.pickup_to_work, tw.dropoff_to_work, fw.pickup_from_work, fw.dropoff_from_work FROM to_work tw JOIN from_work fw ON tw.dropoff_location_to_work = fw.pickup_location_from_work AND tw.pickup_location_to_work = fw.dropoff_location_from_work AND fw.pickup_from_work > tw.dropoff_to_work AND tw.pickup_to_work < tw.dropoff_to_work AND tw.dropoff_to_work < fw.pickup_from_work AND fw.pickup_from_work < fw.dropoff_from_work WHERE ABS(EXTRACT(EPOCH FROM fw.pickup_from_work) - EXTRACT(EPOCH FROM tw.dropoff_to_work)) <= 900;").fetchdf()

# Display the result
print(result)

# Display the first few rows to visually confirm output structure
print(result.head())

# Check if pickup_to_work < dropoff_to_work < pickup_from_work < dropoff_from_work for each row
result['is_sequence_correct'] = (
    (result['pickup_to_work'] < result['dropoff_to_work']) &
    (result['dropoff_to_work'] < result['pickup_from_work']) &
    (result['pickup_from_work'] < result['dropoff_from_work'])
)

# Print rows where the sequence check failed
incorrect_rows = result[~result['is_sequence_correct']]
print("Rows with incorrect timestamp sequence:", incorrect_rows)


             pickup_to_work     dropoff_to_work    pickup_from_work  \
0       2021-08-06 17:28:44 2021-08-06 17:54:04 2021-08-06 17:56:20   
1       2021-08-06 17:00:52 2021-08-06 17:26:58 2021-08-06 17:40:42   
2       2021-08-06 17:59:27 2021-08-06 18:17:42 2021-08-06 18:20:13   
3       2021-08-06 17:52:30 2021-08-06 18:07:06 2021-08-06 18:09:11   
4       2021-08-06 17:36:08 2021-08-06 17:55:28 2021-08-06 18:05:52   
...                     ...                 ...                 ...   
2281405 2021-08-31 18:23:44 2021-08-31 18:32:52 2021-08-31 18:45:24   
2281406 2021-08-31 18:22:32 2021-08-31 18:36:43 2021-08-31 18:51:38   
2281407 2021-08-31 17:57:49 2021-08-31 18:16:46 2021-08-31 18:30:01   
2281408 2021-08-31 18:05:43 2021-08-31 18:12:21 2021-08-31 18:22:03   
2281409 2021-08-31 18:50:27 2021-08-31 18:56:01 2021-08-31 18:56:36   

          dropoff_from_work  
0       2021-08-06 18:30:45  
1       2021-08-06 18:16:29  
2       2021-08-06 18:42:19  
3       2021-08-06 18:26:20

## Feedback (voluntary)

How did you like this exercise? What could be improved?

Answer:

...

Further, I feel like:
 - [ ] the exercise was too easy
 - [X] the exercise was too hard
 - [] the exercise was just right
 - [] no answer
